In [2]:
import os
os.chdir("/project/6105902/shougan/balance-budget")

In [3]:
from tuning.inference.vllm_utils import load_vlm_model, make_vllm_call
from vllm import SamplingParams
from tuning.utils.utils import apply_chat_template, chat_template_func
from tuning.config import IFEVAL_OUTPUTS_DIR, RESPONSES_FILENAME
from tuning.utils.gpt_utils import save_responses
from datasets import load_from_disk, DatasetDict
import numpy as np
import torch


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 12-08 16:50:07 [__init__.py:244] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This m

/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning


In [11]:
def get_response_logp(llm, prompt_messages, response_message=None):
    tokenizer = llm.get_tokenizer()
    tokenizer = chat_template_func(tokenizer) # add chatML
    
    if response_message:
        prompt_str = tokenizer.apply_chat_template(prompt_messages, tokenize=False, add_generation_prompt=True)
        prompt_ids = tokenizer.encode(prompt_str)

        messages_with_response = prompt_messages + response_message
        full_text = tokenizer.apply_chat_template(messages_with_response, tokenize=False, add_generation_prompt=False)
        full_input_ids = tokenizer.encode(full_text)

        sampling_params = SamplingParams(
            prompt_logprobs=100000,   
            max_tokens=1,        
            temperature=0.0,
            stop=[],
        )

        outputs = llm.generate([full_text], sampling_params)
        output = outputs[0]

        prompt_logprobs = output.prompt_logprobs
        
        response_log_probabilities = []
        for i in range(len(prompt_ids), len(full_input_ids)):
            token_i = full_input_ids[i] 
            logprob_dict = prompt_logprobs[i] # each generated token has its own logP dictionary
            
            if token_i in logprob_dict:
                response_log_probabilities.append(logprob_dict[token_i].logprob)
            else:
                print(f"CANNOT FIND THE TOKEN {token_i} at {i}")
                response_log_probabilities.append(-100.0) 

        return response_message, sum(response_log_probabilities)


    else:
        sampling_params = SamplingParams(
            prompt_logprobs = 1,
            temperature=0,
            logprobs=1,
            max_tokens=4096,
        )
        prompt_str = tokenizer.apply_chat_template(prompt_messages, tokenize=False, add_generation_prompt=True)
        outputs = llm.generate(prompt_str, sampling_params)

        output = outputs[0].outputs[0]
            
        response_log_probabilities = []
        for logprob in output.logprobs:
            token_id = list(logprob.keys())[0] # get the id of the single chosen token 
            response_log_probabilities.append(logprob[token_id].logprob)
            
        total_logp = np.sum(response_log_probabilities)

        return output.text, total_logp


In [4]:
model_name = "llama3-8B"


In [8]:
llm, _ = load_vlm_model(model_name, max_logprobs = 100000)

Loading model from /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-8B
Using GPU memory utilization: 0.8
INFO 12-08 12:56:11 [config.py:841] This model supports multiple tasks: {'embed', 'reward', 'classify', 'generate'}. Defaulting to 'generate'.


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 12-08 12:56:11 [config.py:3368] Downcasting torch.float32 to torch.bfloat16.
INFO 12-08 12:56:11 [config.py:1472] Using max model len 131072
INFO 12-08 12:56:14 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 12-08 12:56:15 [__init__.py:2662] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 12-08 12:56:22 [__init__.py:244] Automatically detected platform cuda.
INFO 12-08 12:56:24 [core.py:526] Waiting for init message from front-end.
INFO 12-08 12:56:24 [core.py:69] Initializing a V1 LLM engine (v0.9.2) with config: model='/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-8B', speculative_config=None, tokenizer='/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-8B', skip_tokenizer_init=False

Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  14% Completed | 1/7 [00:01<00:09,  1.62s/it]
Loading safetensors checkpoint shards:  29% Completed | 2/7 [00:03<00:08,  1.68s/it]
Loading safetensors checkpoint shards:  43% Completed | 3/7 [00:05<00:06,  1.73s/it]
Loading safetensors checkpoint shards:  57% Completed | 4/7 [00:06<00:05,  1.76s/it]
Loading safetensors checkpoint shards:  71% Completed | 5/7 [00:08<00:03,  1.76s/it]
Loading safetensors checkpoint shards:  86% Completed | 6/7 [00:10<00:01,  1.77s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:11<00:00,  1.49s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:11<00:00,  1.63s/it]



INFO 12-08 12:56:37 [default_loader.py:272] Loading weights took 11.45 seconds
INFO 12-08 12:56:38 [gpu_model_runner.py:1801] Model loading took 15.0006 GiB and 11.904924 seconds
INFO 12-08 12:56:50 [backends.py:508] Using cache directory: /home/shougan/.cache/vllm/torch_compile_cache/30aefafaa6/rank_0_0/backbone for vLLM's torch.compile
INFO 12-08 12:56:50 [backends.py:519] Dynamo bytecode transform time: 12.04 s
INFO 12-08 12:56:57 [backends.py:155] Directly load the compiled graph(s) for shape None from the cache, took 6.674 s
INFO 12-08 12:56:58 [monitor.py:34] torch.compile takes 12.04 s in total
INFO 12-08 12:56:59 [gpu_worker.py:232] Available KV cache memory: 19.20 GiB
INFO 12-08 12:56:59 [kv_cache_utils.py:716] GPU KV cache size: 157,296 tokens
INFO 12-08 12:56:59 [kv_cache_utils.py:720] Maximum concurrency for 131,072 tokens per request: 1.20x


Capturing CUDA graph shapes: 100%|█████████████| 67/67 [00:18<00:00,  3.66it/s]


INFO 12-08 12:57:18 [gpu_model_runner.py:2326] Graph capturing finished in 18 secs, took 0.56 GiB
INFO 12-08 12:57:18 [core.py:172] init engine (profile, create kv cache, warmup model) took 40.45 seconds


In [5]:
dataset_path = "/project/6105902/shougan/balance-budget/tuning/data/datasets/sft-tuluif"
full_dataset = load_from_disk(dataset_path)
data_point = full_dataset["train"][1]

prompt_message = prompt_message = [data_point['messages'][0], data_point['messages'][1]] #index 0 and 1 is system and user question
ground_truth_answer = [data_point['messages'][2]]
print(prompt_message)

[{'content': 'You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions', 'role': 'system'}, {'content': 'As a financial advisor with expertise in the insurance sector, create an informative guide on "The Evolution of Life Insurance Policies in the United States" for your clients in Brentwood, Tennessee. Your guide should meet the following constraints:\n\n- The guide must be structured into three distinct paragraphs.\n- Within each paragraph, include at least two keywords from the following list: "policyholder," "beneficiary," "underwriting," "premium," "coverage."\n- Ensure that each paragraph contains at least one sentence where the letter \'i\' appears at least 8 times.\n- Conclude your guide with a summary in the form of two bullet points, each point containing the word "insurance" at least once.', 'role': 'user'}]


In [12]:
greedy_answer, greedy_logp = get_response_logp(llm, prompt_message)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                                                 | 0/1 [00:00<?, ?it/s, est. speed inp…

In [13]:
print(greedy_answer)
print(greedy_logp)

The Evolution of Life Insurance Policies in the United States
Life insurance has been a crucial component of financial planning for individuals and families in the United States for over two centuries. Over time, life insurance policies have evolved to meet the changing needs and preferences of policyholders. In this guide, we will explore the history of life insurance policies in the United States, focusing on key milestones and developments that have shaped the industry.

The Early Years: 18th and 19th Centuries
The first life insurance policies in the United States were introduced in the late 18th century. These policies were primarily designed to provide financial protection for families in the event of the death of the breadwinner. The early policies were often simple and straightforward, with a fixed premium and a death benefit paid to the beneficiary upon the policyholder's death.

The 20th Century: Growth and Innovation
The 20th century saw significant growth and innovation in 

In [126]:
ground_answer, ground_logp = get_response_logp(llm, prompt_message, response_message=ground_truth_answer)

NameError: name 'llm' is not defined

In [15]:
print(ground_answer)
print(ground_logp)

[{'content': '**The Evolution of Life Insurance Policies in the United States**\n\nLife insurance has undergone significant changes over the years, adapting to the evolving needs of policyholders and the regulatory environment. Initially, life insurance policies were simple agreements between the insurer and the policyholder, offering basic coverage to ensure financial security for beneficiaries. Over time, the underwriting process has become more sophisticated, allowing insurers to better assess risk and tailor premiums to individual circumstances. This evolution has made life insurance more accessible and affordable, while providing policyholders with more options to suit their specific needs.\n\nThe role of underwriting in life insurance has expanded significantly, with advances in technology enabling insurers to gather and analyze more data than ever before. Underwriting now involves assessing a range of factors, from medical history to lifestyle habits, to determine the appropriat

In [6]:
import torch.nn.functional
from transformers import AutoModelForCausalLM, AutoTokenizer

def get_response_logp_transformers(model, tokenizer, prompt_messages, response_message = False):
    
    if response_message: # get logP of existing response
        prompt_str = tokenizer.apply_chat_template(prompt_messages, tokenize=False, add_generation_prompt=True)

        prompt_ids = tokenizer.encode(prompt_str, return_tensors="pt").to(model.device)

        messages_with_response = prompt_messages + response_message
        full_text = tokenizer.apply_chat_template(messages_with_response, tokenize=False, add_generation_prompt=False)
        full_input_ids = tokenizer.encode(full_text, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            outputs = model(full_input_ids)
            logits = outputs.logits
        response_start_index = len(prompt_ids[0])
        print(f"Shape of prompt_id {prompt_ids.shape}, {response_start_index}")

        relevant_logits = logits[0, response_start_index - 1:-1, :]
    
        response_ids = full_input_ids[0, response_start_index:]
        
        log_probs = torch.nn.functional.log_softmax(relevant_logits, dim=-1) # softmax and log the logits
    
        response_log_probs = []
    
        for i in range(len(response_ids)) :
            token_id_i = response_ids[i].item()
            log_prob_i = log_probs[i, token_id_i].item()
            response_log_probs.append(log_prob_i)
    
        return response_message, sum(response_log_probs)
    else:
        prompt_ids = tokenizer.apply_chat_template(
                        prompt_messages,
                        tokenize=False, 
                        add_generation_prompt = True
        )

        prompt_ids = tokenizer.encode(prompt_ids, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            generated_output = model.generate(
                prompt_ids,
                max_new_tokens=4096,
                num_beams = 1,
                do_sample=False,  # Greedy decoding
                return_dict_in_generate=True,
                output_scores=True # need .score
            )
            
        generated_ids = generated_output.sequences[0, prompt_ids.shape[-1]:] # exclude the prompt_ids
        
        response_log_probs = []
        for i, token_id in enumerate(generated_ids):
            logits_at_i = generated_output.scores[i][0] 
            log_probs_at_i = torch.nn.functional.log_softmax(logits_at_i, dim=-1)
            log_prob_i = log_probs_at_i[token_id].item()
            response_log_probs.append(log_prob_i)
        
        generated_text = tokenizer.decode(generated_ids)
        return generated_text, sum(response_log_probs)
    


In [7]:
torch.cuda.empty_cache()
hf_path = f"/project/6105902/shougan/balance-budget/tuning/models/{model_name}"
hf_model = AutoModelForCausalLM.from_pretrained(hf_path, torch_dtype=torch.bfloat16, device_map="auto")
hf_tokenizer = AutoTokenizer.from_pretrained(hf_path)
hf_tokenizer = chat_template_func(hf_tokenizer)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


In [8]:
ground_response, ground_truth_logp_hf = get_response_logp_transformers(hf_model, hf_tokenizer, prompt_message, response_message = ground_truth_answer)

Shape of prompt_id torch.Size([1, 188]), 188


In [9]:
print(ground_truth_logp_hf)
print(ground_response) 

-451.28200340270996
[{'content': '**The Evolution of Life Insurance Policies in the United States**\n\nLife insurance has undergone significant changes over the years, adapting to the evolving needs of policyholders and the regulatory environment. Initially, life insurance policies were simple agreements between the insurer and the policyholder, offering basic coverage to ensure financial security for beneficiaries. Over time, the underwriting process has become more sophisticated, allowing insurers to better assess risk and tailor premiums to individual circumstances. This evolution has made life insurance more accessible and affordable, while providing policyholders with more options to suit their specific needs.\n\nThe role of underwriting in life insurance has expanded significantly, with advances in technology enabling insurers to gather and analyze more data than ever before. Underwriting now involves assessing a range of factors, from medical history to lifestyle habits, to dete

In [10]:
 greedy_response_hf, greedy_logp_hf = get_response_logp_transformers(hf_model, hf_tokenizer, prompt_message)

In [11]:
print(greedy_logp_hf)
print(greedy_response_hf)

-4.790224567012956
As a financial advisor with expertise in the insurance sector, create an informative guide on "The Evolution of Life Insurance Policies in the United States" for your clients in Brentwood, Tennessee. Your guide should meet the following constraints:

- The guide must be structured into three distinct paragraphs.
- Within each paragraph, include at least two keywords from the following list: "policyholder," "beneficiary," "underwriting," "premium," "coverage."
- Ensure that each paragraph contains at least one sentence where the letter 'i' appears at least 8 times.
- Conclude your guide with a summary in the form of two bullet points, each point containing the word "insurance" at least once.<|im_end|>


In [12]:
prompt_str = hf_tokenizer.apply_chat_template(prompt_message, tokenize = False, add_generation_prompt = True)

In [14]:
print(prompt_str)


<|im_start|>system
You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions<|im_end|>
<|im_start|>user
As a financial advisor with expertise in the insurance sector, create an informative guide on "The Evolution of Life Insurance Policies in the United States" for your clients in Brentwood, Tennessee. Your guide should meet the following constraints:

- The guide must be structured into three distinct paragraphs.
- Within each paragraph, include at least two keywords from the following list: "policyholder," "beneficiary," "underwriting," "premium," "coverage."
- Ensure that each paragraph contains at least one sentence where the letter 'i' appears at least 8 times.
- Conclude your guide with a summary in the form of two bullet points, each point containing the word "insurance" at least once.<|im_end|>
<|im_start|>assistant



In [37]:
prompt_ids = hf_tokenizer.encode(prompt_str, return_tensors="pt").to(hf_model.device)

In [38]:
with torch.no_grad():
    generated_output = hf_model.generate(
        prompt_ids[:,1:], 
        max_new_tokens = 4096, 
        num_beams = 1, 
        do_sample = False, 
        return_dict_in_generate = True, 
        output_scores = True
    )

In [39]:
generated_ids = generated_output.sequences[0, ]
print(generated_output.sequences.shape)

torch.Size([1, 232])


In [40]:
generated_text = hf_tokenizer.decode(generated_ids)

In [41]:
print(generated_text)

<|im_start|>system
You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions<|im_end|>
<|im_start|>user
As a financial advisor with expertise in the insurance sector, create an informative guide on "The Evolution of Life Insurance Policies in the United States" for your clients in Brentwood, Tennessee. Your guide should meet the following constraints:

- The guide must be structured into three distinct paragraphs.
- Within each paragraph, include at least two keywords from the following list: "policyholder," "beneficiary," "underwriting," "premium," "coverage."
- Ensure that each paragraph contains at least one sentence where the letter 'i' appears at least 8 times.
- Conclude your guide with a summary in the form of two bullet points, each point containing the word "insurance" at least once.<|im_end|>
<|im_start|>assistant
You are a skilled assistant who excels at following instructions and meeting constraints. Your guide will 